In [3]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

C:\Users\v-savrav\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
df = pd.concat(pd.read_excel("C:\\Users\\v-savrav\\Documents\\SampleDataset2.xlsx", sheet_name=None), ignore_index=True)

In [5]:
df.columns

Index(['PrimaryTopic', 'secondarytopic', 'description'], dtype='object')

In [6]:
# Convert to list
data = df["description"].values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['hi', 'team', 'please', 'cancel', 'the', 'first', 'submission', 'of', 'er', 'as', 'the', 'incorrect', 'receipts', 'were', 'attached', 'however', 'am', 'now', 'going', 'to', 'submit', 'the', 'correct', 'receipts', 'to', 'the', 'er', 'thank', 'you', 'regards', 'renee', 'bradshaw']]


In [12]:
df['description'][990]

'Hi APAU Team, Please provide the bank statement of 10741948 GIRO Account of June 2017. Best regards,  Sara  86-10-59165616'

In [7]:
df['URN'] = df['description'].str.extract(r"(\W[7][6]\d{6}\W|\W[3][4]\d{6}\W|\W[5][4]\d{6}\W|\W[6][5]\d{6}\W|\W[7][5]\d{6}\W|\W[1][4]\d{6}\W|\W[7][1]\d{6}\W|\W[6][8]\d{6}\W|\W[6][6]\d{6}\W|\W[6][3]\d{6}\W|\W[6][0]\d{6}\W|\W[1][6]\d{6}\W)")
df['PO'] = df['description'].str.extract(r'(\W[9][8]\d{6}\s|\W[9][9]\d{6}\s|\W[9][7]\d{6}\s|\W[8][0]\d{6}\s|\W[7][0]\d{6}\s|\W[7][1]\d{6}\s|\W[1][0]\d{6}\s|\W[4][7]\d{7}\s|\W[4][1]\d{8}\s|\W[6][1]\d{8}\s|\W[6][0]\d{8}\s)', expand=True)

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'check','apars_apar','error'])
stop_words.extend(['close','let','hour','know','good','need','subject','desk','find', 'name','thank','thanks','send','com','prohelp','regards','regard','information','message','team',
                   'mail','check','apars_apar','error','attached','www','hi','cc','sent','date','use','try','query','intend','contain','recipient','request','hour','number'])

In [14]:
def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [15]:
def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc ]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load("en_core_web_sm")
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words_nostops)

print(data_lemmatized[:2])

['please cancel first submission er incorrect receipt however go submit correct receipt er renee bradshaw', 'ap please discard email po urn poboxfrom po box nfe friday june pm gunaseelan jayaraj gujaya microsoft po box nfe nfebr microsoft fw regard po ap follow po pobox po box nfe friday june pm angela domingues moreira adecco recursos humanos angdom microsoft fw regard po segue po correta obrigadodanilo gimenez customer service brazil email danilo docuprint br office skype dgimenez dek web docuprint address avenida fagundes de oliveira unidade piraporinha cep diadema sao paulo de po box nfe mailto nfebr microsoft enviada  sexta feira de junho de para jean carlos valeriano de araujo danilo docuprint br po box nfe nfebr microsoft assunto fw regard po danilo boa tarde po informada para fatura  anexo nao valida por gentileza informar po correta att pobox po box nfe thursday june pm angela domingues moreira adecco recursos humanos angdom microsoft fw regard po po gunaseelan jayaraj thursda

In [34]:
Vendor_master = ["microsoft payment central","Payment central","APMDG","AP MDG","CR #","change request","MDG" ,"bank account chnag","new vendor creation" ]#,payment terms change ,vendor block,vendor email/fax,request ID,MDG system,MPC system,MPC tool,Bank Key update,
#invoice_payment = ["URN","PO","Purchase order","reference number","enclosed","invoice","attached invoice","TDS" ,"rectification" ,"With hold tax","staus of invoice","enclosed","remitance advice","payemt status","MS invoice Tool","POE","PO" ]#extension#,MS invoice#,Attached EI file,EI upload,remittance,partner invoice,microsoft PO,vendor ID,invoice #,reference #,following invoice #,statement of account,upload the attached file,check #,cheque #,DD #,Tax payment ,supplier #,
Expense_Report= ["emp number" ,"reimbursement","last WorkingDay ,Final Working day,EOS,End of settlement,er ,expense,MS authorize,Audit" ,"final status of claim","submitting my claim","submiting my expense","MS expense"]#, "MS expense 1.0"]#, MS expense 2.0,MS expense ,MyExpense,Expense Report,Credit Card payment,My card payment,Meal Expense,Flight Charge,Flight expense,Last Travel expense,Meals,Audit,Audit inquiries,Airport drop,T&E,Amex Card payment,TE
#Disbursement = ["Failed payment","rejected","citi bankquery,refund entry,bank GL clearning,proof of payment,reverse 1500, idoc,run id,citibank account,bank rejection, CI refund,bank rejection reason,payment return,rejection reason,custom duty,bank GL posting


In [46]:
invlist=['MS Invoice Number','Invoice No.','Invoice #','Invoice Number','Invoice No','Vendor Invoice Number','Invoice#','Invoice reference',
            'Tax invoice number','MS Doc #','Invoice No','Invoice','Microsoft Invoice', 'Invoice reference','INVOICE#','Document Number',
           'Invoice NO. or Descriptions','po','urn','Reference Number','REFERENCE','Microsoft reference','PO Number','Master PO No','Po #','ECIF PO NO',"URN","PO","Purchase order","reference number","enclosed","invoice","attached invoice","TDS" ,"rectification" ,"With hold tax","staus of invoice","enclosed","remitance advice","payemt status","MS invoice Tool","POE","PO" ]#extension#,MS invoice#,Attached EI file,EI upload,remittance,partner invoice,microsoft PO,vendor ID,invoice #,reference #,following invoice #,statement of account,upload the attached file,check #,cheque #,DD #,Tax payment ,supplier #,
df_Topic = pd.DataFrame()
for long_string in data_lemmatized:
    #if df
       #for number in list:
    #print(long_string)
    if re.compile('|'.join(invlist),re.IGNORECASE).search(long_string): 
        print(long_string)
        #re.IGNORECASE is used to ignore case
        df_Topic == "Invoice_Payment"
        print('----------')
        print("Invoice_Payment")
    elif re.compile('|'.join(Vendor_master),re.IGNORECASE).search(long_string): 
        print(long_string)
        #re.IGNORECASE is used to ignore case
        df_Topic == "Vendor_master"
        print('----------')
        print("Vendor_master")
    elif re.compile('|'.join(Expense_Report),re.IGNORECASE).search(long_string): 
        print(long_string)
        #re.IGNORECASE is used to ignore case
        print('----------')
        print("Expense_Report")
        df_Topic == "Expense_Report"
       
        
      
       

ap please discard email po urn poboxfrom po box nfe friday june pm gunaseelan jayaraj gujaya microsoft po box nfe nfebr microsoft fw regard po ap follow po pobox po box nfe friday june pm angela domingues moreira adecco recursos humanos angdom microsoft fw regard po segue po correta obrigadodanilo gimenez customer service brazil email danilo docuprint br office skype dgimenez dek web docuprint address avenida fagundes de oliveira unidade piraporinha cep diadema sao paulo de po box nfe mailto nfebr microsoft enviada  sexta feira de junho de para jean carlos valeriano de araujo danilo docuprint br po box nfe nfebr microsoft assunto fw regard po danilo boa tarde po informada para fatura  anexo nao valida por gentileza informar po correta att pobox po box nfe thursday june pm angela domingues moreira adecco recursos humanos angdom microsoft fw regard po po gunaseelan jayaraj thursday june po box nfe nfebr microsoft regard po kindly provide valid po invoice gunaseelan
----------
Invoice_Pay

receive submit invoice po would please submit invoice poe case issue concern vendor invoice susmita ghosh accenture avanade cfm email susmita ghosh accenture accenture avanade designate may privilege proprietary otherwise confidential receive please notify sender immediately delete original prohibit allow local law electronic communication accenture affiliate include instant messaging include content may scanned system purpose security assessment internal compliance accenture policy accenture
----------
Invoice_Payment
invoice th time apology frustration please prioritize andres ms ireland account payable friday june ebru erol yildirim consulting meerol microsoft andres valencia andres valencia microsoft paul schuman pschuman microsoft paul schuman pschuman microsoft tkt fw invoice ebru reference invoice copy email contact supplier provide invoice copy open amount process us query kiran microsoft one finance europe account payable feel get response within acceptable timeframe inadequat

supplier contact clear could please clarify provide details luis rubi hernandez garcia radiomovil dipsa sa de cv friday june pm luis corona luis corona microsoft fw attention require invoice approver approve time period hola luis te reenvio el correo de por una factura pendiente del quedo atenta tus comentarios saludos cordiale rubi hernandez plant telcel microsoft informacion soporte de telefonia celularemail microsoft vmtsupport tangoe comapertura de incidentes en linea visita voice management toolsoporte telefonico para problemas con tu plan de telefonia soporte de roam internacional esta disponible los dias desde fijo las horas del dia eu desde fijo desde tu telcel canada desde fijo problemas con lineas de datos helpdesk datos helpdesk_datos telcel tel de lunes domingo las hrs de gudipati accenture mailto mangud microsoft enviado el jueves de junio de para support platam microsoft claudia austria garcia clausgar telcel asunto attention require invoice approver approve time period d

sidnei see msinvoice well invoice approve process ms brazil ap help  understand status po ok georgia takashima microsoft brasil epg education getaka microsoft sidnei de castilho drobnievski mailto sidnei bigbrain br friday june pm georgia takashima getaka microsoft enc please disregard workshop office aqui diz que esta okay de fy adoption offer mailto svcadopt microsoft enviada  friday june pm para ronei pasquetto ronei bigbrain br sidnei de castilho drobnievski sidnei bigbrain br marelyn magda marquett marelyn bigbrain br fy adoption offer svcadopt microsoft assunto please disregard workshop office ronei submit invoice still pende approve tool open month end stefanie fy adoption offer payment email svcadopt microsoft fy ems cloud pbx adoption buye offer officially june invoice work complete must submit later june ensure poe collection approval processing prior fiscal hour operation monday friday pacific time redmond washington usa standard response time inquiry within hour base post h

dear sender email scanning philipine inv ms edi accenture process successfully document split pdf split pdf assign follow unique reference urn urn query please contact amt vps helpdesk accenture urn kind designate may privilege proprietary otherwise confidential receive please notify sender immediately delete original prohibit allow local law electronic communication accenture affiliate include instant messaging include content may scanned system purpose security assessment internal compliance accenture policy accenture
----------
Invoice_Payment
dear sender email fw tkt fw ap telemar norte leste philipine inv ms edi accenture process successfully document fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb certo pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi bsb pdf fy oi b

...

hello razia kindly share report sowbar sadhik accenture microsoft one finance nepa records accenture outsource support microsoft onefinance chennai one one vision onefinance razia raja mohammed accenture thursday june pm ms lebanon record reclb microsoft bharathi ramani accenture bhrama microsoft jothi prakasam accenture jothip microsoft ragul ganesh accenture ragga microsoft chandrasekaran ramasamy chram microsoft velmurugan annamalai accenture velann microsoft deepika mahapatra accenture demaha microsoft kusumeswar katreddi accenture kukatr microsoft ramyaa thenraj rathen microsoft dummy vat report lebanon hello dummy vat report share rd july since capture transaction belong razia account payable global disbursement mea nepa microsoft onefinance accenture outsource chennai india check invoice status expense report status msinvoice urgent assistance cim one stop portal help pos invoice expense payment query internal microsoft ms mea accounts payable commit respond within hour sla meet

please zero report day arun pandian paarun microsoft accounts payable accenture outsource support microsoft onefinance chennai india one one vision onefinance save paper save tree switch go recycle always
----------
Invoice_Payment
anitha may ask update swift jennifer valeza mbs escalation manager crm premise ax microsoft dynamics csa fee original ci pay account payable receive naoccie naoccie escalation anitha sarah mccullough sarah mccullough sarah mccullough saramc microsoft tkt fw csa payments crm jennifer request bank swift share upon response anitha ap disbursement one finance accenture outsource support microsoft onefinance chennai india ci payment monitoring commit respond within business hour sla meet response inadequate please escalate sashik microsoft onefinance one one vision one finance please print email unless absolutely necessary original naoccie microsoft receive tue jun gmt india standard time ci payment monitoring sarah mccullough sarah mccullough fw csa payments crm

sunil add ap help ap kindly help airtel windows phone sunil gupta july behram daruwalla randstad india ltd site operation gsmo marek simm motiv inc sasikumar balasubramanian aman zindal reg provide pan gst pri microsoft corporation india pvt ltd dear behram greeting airtel discuss mr girish kindly provide pan gst pri microsoft corporation india pvt ltd instal jaipur rajasthan billing address gurgaon mention microsoft corporation india pvt ltd po level tower dlf epitome building dlf cyber city phase iii gurgaon har good sunil gupta corporate services airtel business sunil gupta airtel bharti airtel limit sunny house malviya marg scheme jaipur image png email file transmit confidential intend solely individual entity address receive email please notify system manager contain confidential intend individual name name addressee disseminate distribute copy please notify sender immediately receive mistake delete system intend notify disclose copying distribute take action reliance content
---

engage partner work rejection change account however vague us communicate partner please provide detail comp code bl business partner parent bl geo bl contract account bl program bl cur bl sap invoice bl payment status bl current clearing doc clearing doc first attempt compuevolucion de mexico cld tr earning usd pende clarification compuevolucion de mexico csp tr earning usd pende partner update pamela villalobos microsoft online partner incentive ocilatam email address pamriv microsoft direct
----------
Invoice_Payment
hello engage partner work rejection change account however vague us communicate partner please provide detail corresponds partner aura consultoria corporativa mpn  would wait response good jerlin hernandez work microsoft channel incentive esa online incentive csp azure sell usage syndication
----------
Invoice_Payment
hello employee employee lam bonnie notify payment microsoft amount usd amount return due rejection reason acc can not locate payment wire deposit banking 

lydia li notification details branch hong kong citibank branch code transaction value transaction type credit threshold great hkd transaction amount hkd account account familiar microsoft datacenter holdings reference customer reference order te amm hk limited please reply address monitor incoming citidirect inquiry direct service representative sincerely citibank customer service
----------
Invoice_Payment
ms senegal ap track emmanuel please copy ap alias track dinesh please provide update confirm skanda kumar kannan chennai delivery center accenture microsoft onefinance emmanuel thiakane smart rh monday july pm dinesh kumar jayaram accenture dijay microsoft kusumeswar katreddi accenture kukatr microsoft skanda kumar kannan accenture skkann microsoft demande paiement irvm senegal xls dinesh please assist senegal cit payment due nondeductible expense show cit return irvm sur reintegrations rgds emmanuel
----------
Invoice_Payment
screenshots chinese partner receive azure payment sap sh

In [45]:
df_Topic = pd.DataFrame()
for long_string in data_lemmatized:
    if re.compile('|'.join(Expense_Report),re.IGNORECASE).search(long_string): 
        print(long_string)
        #re.IGNORECASE is used to ignore case
        print('----------')
        print("Expense_Report")
        df_Topic == "Expense_Report"

hello file ready process total reimbursement recreational league fee reimbursement bettina
----------
Expense_Report
hello please docusign link siva sai tej pasupuleti emp sai send link form please complete field box red field require update employee expense reimbursement account accurately addition please bolde reply related field formcompany code employee set extend relocation reimbursement mgr reimbursement relocationif relocation relocation contact yvstac microsoft comfor relocation reimbursement currency pay please mention currency code wish reimburse iban bank please add bank field section docusign form ensure employee  would correct good wish yvette yvette stachowiak relocation specialist cartus global mobility services microsoft way attn global mobility ne th st bldng ste bellevue wa yvstac microsoft cartus tel fax attn yvstac microsoft office hour pm pst connect us cartus facebook youtube twitter linkedin celebrate year trust guidance please note cartus office close independen

document ms asia pacific technology ap zhenghui yan zhyan microsoft na qu accenture naq microsoft tkt set expense reimbursement account update expense account please login cim submit notice help fill receive yet inquiry please feel free contact  best na qu naq microsoft account payable china ap chn microsoft account payable korea apkr microsoft account payable taiwan aptw microsoft vendor master file vmf accounts payable accenture outsource support microsoft onefinance one one vision one finance together well account payable commit respond within business hour sla meet response inadequate please escalate tier account payable vmf lead lili yin liyin microsoft tier account payable lead chunmei wang wangch microsoft original zhenghui yan receive thu jun gmt china standard time ms asia pacific technology ap set expense reimbursement account msexpense follow expense report could process payment expense reimbursement account setup please follow local subsidiary process set expense reimbursem

duplicate kindly set era foreign inter pan exception confirm asap simi radhakrishnan microsoft research ph niveditha spectrum consultants india pvt monday july pm simi radhakrishnan spectrum consultants india pvt simir microsoft msr india hr msrindhr microsoft fw expense reimbursement data form wol simi please help get exception wonyeol lee set era without pan apply niveditha wonyeol lee monday july pm msr india hr msrindhr microsoft deevarani baskaran accenture debask microsoft ms india ap smsg apinsms microsoft expense reimbursement data form wol hr era form reject lack pan could make exception apply pan week ago still wait receive wonyeol deevarani baskaran accenture monday july pm wonyeol lee wol microsoft ms india ap smsg apinsms microsoft expense reimbursement data form wol wonyeol rejection notification use expense reimbursement data form service unfortunately able process reject reject pan complete correctly resubmit new order  complete change request note pan mandatory create 

hello pro help do exactly erwin say everything process still see charge unreconciled mariah moon dunn monday june pm microsoft kelli pennylegion kelli pennylegion microsoft tkt expense report question kelli please look ok actual reimbursement payment need one monday june pm kelli pennylegion kelli pennylegion microsoft mariah moon dunn madun microsoft mariah moon dunn madun microsoft tkt expense report question hello kelli mariah email show expense er manually enter create pocket expense report pay directly since expense originally submit pocket reimbursement pay pay amex directly unreconciled charge submit expense expense report use expense category reimbursement comment explain charge submit pocket er use reimbursement pay amex directly erwin microsoft one finance  account payable feel get response within acceptable timeframe inadequate response please escalate  ap escalation usapesc microsoft note please quote ticket numbertkt mention line future reference original kelli pennylegion

joy please advise credit balance refund amex card amount refund sgd cheque payee cheque option available mailing address good serene japa dedicated account manager microsoft american express corporate card global client group global corporate payment american express international inc ms singapore ap thursday june pm joy quek joyquek microsoft ms asia corp card msasiacc microsoft tkt fw employee expense reimbursement amex email please inform expense report relate march may month amex transaction amount sgd submit post per new process amount process microsoft end amex direct payment process pay amex account however expense already pay employee end may month statement balance payback credit amount sgd submit expense report er employee bank account remittance amount receive microsoft joy share expense detail expense report request amex payback process amount submit expense report bank account us clarification glad assist karthick baskaran microsoft one finance account payable chennai indi

nancy use expense reimbursement data form service glad inform expense account create system active submit expense report manager approval please refer screen shot reference employee  would employee nancy abi abdallah us concern dhivya perumal account payable ap global vmf microsoft one finance accenture outsource chennai india ms ap commit respond within business hour sla meet response inadequate may escalate mea apmea microsoft mohammed ismayl accenture sunday july pm dhivya perumal accenture dhperu microsoft ms uae dubai ap apae microsoft ap mea escalation apmea microsoft nancy abi abdallah nancy abdallah microsoft amine amri aamri microsoft tkt expense claim dhiya discuss could please help assist email urgent ismayl microsoft one finance mea account payable feel get response within acceptable timeframe inadequate response please escalate ap mea escalation apmea microsoft sla hour check invoice status expense report status ms invoice urgent assistance cim one stop portal help pos inv

serene please reply inline cheers joy ms asia corp card saturday july pm ms singapore ap apsg microsoft joy quek joyquek microsoft tkt fw employee expense reimbursement joy please advise credit balance refund amex card amount refund sgd cheque payee cheque option available joy quek chwee ying mailing address jalan tenteram good serene japa dedicated account manager microsoft american express corporate card global client group global corporate payment american express international inc ms singapore ap thursday june pm joy quek joyquek microsoft ms asia corp card msasiacc microsoft tkt fw employee expense reimbursement amex email please inform expense report relate march may month amex transaction amount sgd submit post per new process amount process microsoft end amex direct payment process pay amex account however expense already pay employee end may month statement balance payback credit amount sgd submit expense report er employee bank account remittance amount receive microsoft joy 

attention ap please see transaction populate ms expense tool mr steven miller list unreconciled amount date back please advise kind hailey allen group administrator smsp mob email haallee microsoft
----------
Expense_Report
work amex regina visibility ms expense old charge rope ap assist remove line item please advise transaction best serene japa dedicated account manager microsoft american express corporate card global client group global corporate payment american express international inc regina jagiello monday july pm ms asia corp card msasiacc microsoft expense show msexpensse check recent transaction reconcile however charge way back properly expensed could please advise wait reconcile regina ms asia corp card thursday june pm regina jagiello rejag microsoft expense show msexpensse regina patience delay datum get loaded system transaction load expect transaction today tomorrow best serene japa dedicated account manager microsoft american express corporate card global client group

marked expense reimbursement other amount already claim cash please guide wish receive payment anindita expense action notification july anindita ghatak hr anindita ghatak microsoft expense report anindita ghatak inr approve final action require time follow expense report approve final approver alias irag process payment expense report er expense owner anindita ghatak expense purpose hr hand gurgaon expense report total inr unattended alias reply forward alias msexpense employee expense reimbursement policy help
----------
Expense_Report
amex travel ms india ap smsg submit charge incur amex card air ticket trip code vkk ms expense er trip schedule july since charge incur time book june expense per guideline expense incur fy submit last week june later june trip cancel reimburse amount credit reimbursement account pay back microsoft deposit cheque somewhere ranga microsoftindia mailto microsoftindia aexp tuesday june pm rangarajan srirangam rangas microsoft amex travel india amextrvl mi

hello employee employee mohammed imran notify payment microsoft amount usd amount return due rejection reason beneficiary payment wire deposit banking details bank citibank na account xxxxx route swift bic citius go tohttps directdeposit deselect toolgo back tool https directdeposit reimbursement checkboxsave please allow hour account update tool allow action thenyou may delete line item add also note employee payroll manage ms payroll employee reimbursement manage ms accounts payable update affect expense reimbursement account arun pandian microsoft one finance  account payable feel get response within acceptable timeframe inadequate response please escalate  ap escalations usapesc microsoft check invoice status expense report status msinvoice urgent assistance cim one stop portal help pos invoice expense payment query internal microsoft cashpro notifications mailto notifications baml friday june pm us disbursement fgodisb microsoft cashpro notification incoming wire confirmation acti

In [28]:
for long_string in data_words_nostops:       
    df_ll=[c for c in long_string if c in invlist]
    
    print(df_ll)

[]
['po', 'urn', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po', 'po']
[]
['po', 'po']
[]
['po']
[]
['po', 'po']
[]
[]
['po']
[]
[]
['po']
[]
['po']
['po', 'po']
['po']
['po', 'po', 'po']
[]
[]
[]
['po']
[]
['po', 'po', 'po']
[]
[]
['po', 'po']
[]
[]
[]
['po']
[]
[]
[]
[]
['po', 'po', 'po', 'po']
[]
['po']
['po', 'po']
[]
[]
['po']
['po']
[]
[]
[]
['po', 'po']
[]
['po']
['po', 'po']
['po']
['po']
[]
['urn', 'urn', 'urn']
['urn', 'urn', 'urn']
['po', 'po']
['po']
['po', 'po', 'po']
['po', 'po', 'po']
[]
['po']
[]
[]
[]
[]
['po']
[]
['po']
[]
[]
['po', 'po', 'po', 'po', 'po', 'po', 'po']
['po', 'po', 'po', 'po']
['po', 'po', 'po', 'po']
['po', 'po', 'po']
['po', 'po', 'po']
['po']
['po']
['po']
[]
[]
[]
[]
[]
[]
[]
['po', 'po']
['po']
['po']
['po', 'po', 'po', 'po', 'po', 'po', 'po']
['po']
[]
[]
[]
[]
['po', 'po']
['po']
[]
['po']
[]
['po']
[]
['po', 'po', 'po', 'po', 'po']
['po', 'po', 'po']
[]
['po']
[]
['po']
['po']
['po', 'po', '

In [27]:
for c in data_words_nostops:
        print(c)
        print("-----")

['please', 'cancel', 'first', 'submission', 'er', 'incorrect', 'receipts', 'however', 'going', 'submit', 'correct', 'receipts', 'er', 'renee', 'bradshaw']
-----
['ap', 'please', 'discard', 'email', 'po', 'urn', 'poboxfrom', 'po', 'box', 'nfe', 'friday', 'june', 'pm', 'gunaseelan', 'jayaraj', 'gujaya', 'microsoft', 'po', 'box', 'nfe', 'nfebr', 'microsoft', 'fw', 'regarding', 'po', 'ap', 'follows', 'po', 'pobox', 'po', 'box', 'nfe', 'friday', 'june', 'pm', 'angela', 'domingues', 'moreira', 'adecco', 'recursos', 'humanos', 'angdom', 'microsoft', 'fw', 'regarding', 'po', 'segue', 'po', 'correta', 'obrigadodanilo', 'gimenez', 'customer', 'service', 'brazil', 'email', 'danilo', 'docuprint', 'br', 'office', 'skype', 'dgimenez', 'dek', 'web', 'docuprint', 'address', 'avenida', 'fagundes', 'de', 'oliveira', 'unidade', 'piraporinha', 'cep', 'diadema', 'sao', 'paulo', 'de', 'po', 'box', 'nfe', 'mailto', 'nfebr', 'microsoft', 'enviada', 'em', 'sexta', 'feira', 'de', 'junho', 'de', 'para', 'jean', 

['kiran', 'sorry', 'understand', 'mean', 'mentioned', 'submitting', 'ei', 'file', 'invoice', 'lauren', 'tolley', 'global', 'execution', 'manager', 'advertising', 'operations', 'global', 'execution', 'simplify', 'optimize', 'deliver', 'office', 'latoll', 'microsoft', 'comfrom', 'ms', 'ireland', 'accounts', 'payable', 'friday', 'june', 'jen', 'krag', 'jennkrag', 'microsoft', 'jen', 'krag', 'jennkrag', 'microsoft', 'jen', 'krag', 'jennkrag', 'microsoft', 'lauren', 'tolley', 'latoll', 'microsoft', 'cindy', 'bondoc', 'arvato', 'digital', 'services', 'llc', 'ciruiz', 'microsoft', 'cindy', 'bondoc', 'arvato', 'digital', 'services', 'llc', 'ciruiz', 'microsoft', 'tkt', 'tkt', 'urgent_ei', 'upload_', 'crm', 'lauran', 'email', 'kindly', 'informed', 'region', 'code', 'depend', 'upon', 'invoice', 'copies', 'provided', 'process', 'region', 'mentioned', 'process', 'region', 'code', 'us', 'queries', 'kiran', 'microsoft', 'one', 'finance', 'europe', 'accounts', 'payable', 'feel', 'getting', 'response'

['hello', 'three', 'purchase', 'orders', 'invoices', 'tied', 'year', 'old', 'invoices', 'show', 'queue', 'approver', 'copied', 'search', 'invoices', 'msinvoice', 'show', 'local', 'finance', 'india', 'alerted', 'issue', 'hoping', 'also', 'look', 'pos', 'closed', 'tried', 'open', 'morning', 'unable', 'provide', 'insight', 'happening', 'invoices', 'invoices', 'paid', 'please', 'open', 'pos', 'kimball', 'garvin', 'finance', 'manager', 'cct', 'hr', 'finance', 'bldg', 'kimball', 'garvin', 'microsoft', 'kristen', 'mattie', 'costales', 'wednesday', 'june', 'pm', 'kimball', 'garvin', 'kimball', 'garvin', 'microsoft', 'pending', 'invoices', 'gsmo', 'hr', 'pos', 'old', 'pos', 'pending', 'invoices', 'yet', 'approved', 'going', 'back', 'help', 'microsoft', 'regional', 'sales', 'corporatio', 'microsoft', 'regional', 'sales', 'corporatio', 'microsoft', 'regional', 'sales', 'corporatio']
-----
['adjunto', 'factura', 'proveedor', 'orden', 'nº', 'aun', 'ingreso', 'cliente', 'banco', 'de', 'la', 'provinc

['supplier', 'contacting', 'clear', 'could', 'please', 'clarify', 'provide', 'details', 'luis', 'rubi', 'hernandez', 'garcia', 'radiomovil', 'dipsa', 'sa', 'de', 'cv', 'friday', 'june', 'pm', 'luis', 'corona', 'luis', 'corona', 'microsoft', 'fw', 'attention', 'required', 'invoice', 'approver', 'approved', 'time', 'period', 'hola', 'luis', 'te', 'reenvio', 'el', 'correo', 'de', 'por', 'una', 'factura', 'pendiente', 'del', 'quedo', 'atenta', 'tus', 'comentarios', 'saludos', 'cordiales', 'rubi', 'hernandez', 'plant', 'telcel', 'microsoft', 'informacion', 'soporte', 'de', 'telefonia', 'celularemail', 'microsoft', 'vmtsupport', 'tangoe', 'comapertura', 'de', 'incidentes', 'en', 'linea', 'visita', 'voice', 'management', 'toolsoporte', 'telefonico', 'para', 'problemas', 'con', 'tu', 'plan', 'de', 'telefonia', 'soporte', 'de', 'roaming', 'internacional', 'esta', 'disponible', 'los', 'dias', 'desde', 'fijo', 'las', 'horas', 'del', 'dia', 'eu', 'desde', 'fijo', 'desde', 'tu', 'telcel', 'canada',

['ap', 'hope', 'finds', 'well', 'please', 'process', 'ei', 'file', 'advised', 'submit', 'invoices', 'via', 'email', 'time', 'matt', 'gray', 'intelligent', 'audit', 'office', 'ext', 'email', 'matt', 'confidentiality', 'notice', 'email', 'including', 'attachments', 'contains', 'may', 'confidential', 'privileged', 'intended', 'individual', 'entity', 'named', 'intended', 'aware', 'disclosure', 'copying', 'distribution', 'contents', 'prohibited', 'received', 'please', 'notify', 'sender', 'immediately', 'return', 'email', 'destroy', 'copies', 'original']
-----
['hello', 'please', 'get', 'status', 'update', 'ms', 'invoice', 'cashed', 'please', 'issue', 'via', 'overnight', 'delivery', 'address', 'kimberly', 'bolte', 'archway', 'accountant', 'accounting', 'services', 'finance', 'operations', 'office', 'kiferc', 'microsoft', 'please', 'take', 'minute', 'provide', 'feedback', 'accounting', 'services', 'satisfaction', 'survey', 'mark', 'pantaleon', 'accenture', 'thursday', 'june', 'pm', 'fargo', '

...

['sunil', 'adding', 'ap', 'help', 'ap', 'kindly', 'help', 'airtel', 'windows', 'phone', 'sunil', 'gupta', 'july', 'behram', 'daruwalla', 'randstad', 'india', 'ltd', 'site', 'operations', 'gsmo', 'marek', 'simm', 'motiv', 'inc', 'sasikumar', 'balasubramanian', 'aman', 'zindal', 'reg', 'provide', 'pan', 'gst', 'pri', 'microsoft', 'corporation', 'india', 'pvt', 'ltd', 'dear', 'behram', 'greetings', 'airtel', 'discussed', 'mr', 'girish', 'kindly', 'provide', 'pan', 'gst', 'pri', 'microsoft', 'corporation', 'india', 'pvt', 'ltd', 'installed', 'jaipur', 'rajasthan', 'billing', 'address', 'gurgaon', 'mentioned', 'microsoft', 'corporation', 'india', 'pvt', 'ltd', 'po', 'level', 'tower', 'dlf', 'epitome', 'building', 'dlf', 'cyber', 'city', 'phase', 'iii', 'gurgaon', 'har', 'best', 'sunil', 'gupta', 'corporate', 'services', 'airtel', 'business', 'sunil', 'gupta', 'airtel', 'bharti', 'airtel', 'limited', 'sunny', 'house', 'malviya', 'marg', 'scheme', 'jaipur', 'image', 'png', 'email', 'files', '

['joseph', 'would', 'like', 'follow', 'update', 'vendor', 'id', 'following', 'bank', 'technical', 'reason', 'rejection', 'claim', 'partner', 'id', 'org', 'program', 'company', 'code', 'vendor', 'id', 'cdi', 'computer', 'dealers', 'inc', 'named', 'oem', 'incentives', 'kamille', 'joy', 'cartago', 'infosys', 'bpo', 'ltd', 'kamcar', 'microsoft', 'working', 'aoc', 'oem', 'channel', 'incentives', 'payments', 'specialist', 'vector', 'building', 'northgate', 'cyberzone', 'filinvest', 'corporate', 'city', 'alabang', 'muntinlupa', 'city', 'metro', 'manila', 'philippines', 'chip', 'finance', 'tuesday', 'june', 'pm', 'kamille', 'joy', 'cartago', 'infosys', 'bpo', 'limited', 'kamcar', 'microsoft', 'berna', 'mae', 'bilbao', 'infosys', 'bpo', 'limited', 'bebilb', 'microsoft', 'berna', 'mae', 'bilbao', 'infosys', 'bpo', 'limited', 'bebilb', 'microsoft', 'berna', 'mae', 'bilbao', 'infosys', 'bpo', 'limited', 'bebilb', 'microsoft', 'carina', 'apetrior', 'infosys', 'bpo', 'limited', 'caapet', 'microsoft'

['could', 'payment', 'make', 'karen', 'jara', 'layme', 'analista', 'ssff', 'anexo', 'kjara', 'ripley', 'pe', 'av', 'las', 'begonias', 'san', 'isidro', 'de', 'platam', 'mailto', 'platam', 'microsoft', 'enviado', 'el', 'lunes', 'de', 'junio', 'de', 'para', 'diana', 'cajusol', 'inonan', 'karen', 'jara', 'layme', 'pamela', 'polo', 'juan', 'piccini', 'lizares', 'asunto', 'tkt', 'rebate', 'dic', 'microsoft', 'karen', 'please', 'note', 'make', 'payment', 'invoice', 'mistake', 'missed', 'pay', 'invoice', 'amount', 'repay', 'balance', 'amount', 'us', 'queries', 'dinesh', 'arokiaraj', 'accenture', 'microsoft', 'one', 'finance', 'accounts', 'payable', 'accounts', 'payable', 'committed', 'respond', 'within', 'hours', 'sla', 'met', 'response', 'inadequate', 'please', 'escalate', 'tier', 'latam', 'escalations', 'pesc', 'microsoft', 'sla', 'business', 'hours', 'checking', 'invoice', 'status', 'expense', 'report', 'status', 'msinvoice', 'urgent', 'assistance', 'cim', 'one', 'stop', 'portal', 'help', '

['estimados', 'senores', 'confirmarian', 'por', 'favor', 'si', 'el', 'efectivo', 'en', 'el', 'er', 'de', 'abajo', 'ya', 'aprobado', 'por', 'el', 'final', 'approver', 'sera', 'incluido', 'en', 'la', 'proxima', 'corrida', 'de', 'pagos', 'muchas', 'gracias', 'clara', 'maria', 'bozzo', 'expense', 'action', 'notification', 'monday', 'july', 'pm', 'clara', 'maria', 'bozzo', 'cela', 'cbozzo', 'microsoft', 'expense', 'report', 'clara', 'maria', 'bozzo', 'ars', 'approved', 'final', 'action', 'required', 'time', 'following', 'expense', 'report', 'approved', 'final', 'approver', 'alias', 'aanderli', 'processed', 'payment', 'expense', 'report', 'er', 'expense', 'owner', 'clara', 'maria', 'bozzo', 'expense', 'purpose', 'sport', 'activity', 'reimbursement', 'expenses', 'expense', 'report', 'total', 'ars', 'unattended', 'alias', 'reply', 'forward', 'alias', 'msexpense', 'employee', 'expense', 'reimbursement', 'policy', 'help']
-----
['lydia', 'li', 'notification', 'details', 'branch', 'hong', 'kong',

-----
['received', 'notification', 'citi', 'payment', 'rejected', 'payment', 'debited', 'payment', 'company', 'code', 'company', 'code', 'microsoft', 'ireland', 'division', 'ms', 'payment', 'issue', 'reject', 'return', 'reject', 'return', 'reject', 'reason', 'invalid', 'beneficiary', 'iban', 'beneficiary', 'entserv', 'schweiz', 'gmbh', 'partner', 'hewlett', 'packard', 'schweiz', 'gmbh', 'bp', 'ca', 'program', 'advisor', 'deploy', 'earning', 'mpn', 'id', 'bp', 'amount', 'currency', 'chf', 'payment', 'document', 'support', 'contact', 'kajam', 'karthika', 'accounts', 'payable', 'ci', 'payment', 'monitoring', 'microsoft', 'one', 'finance', 'accenture', 'outsourcing', 'ci', 'payment', 'monitoring', 'committed', 'respond', 'within', 'hours', 'sla', 'met', 'response', 'inadequate', 'may', 'escalate', 'sashi', 'kumarv', 'sashik', 'microsoft']
-----


In [ ]:
for email in data_lemmatized:
    #for word in email:
    for word in email :
        print(word)
        print("-----------------------------")

In [46]:
def check(sentence, words): 
    res = [all([k in s for k in words]) for s in sentence] 
    return [sentence[i] for i in range(0, len(res)) if res[i]] 
      
# Driver code 
sentence = ['python coder', 'geeksforgeeks', 'coder in geeksforgeeks'] 
words = ['coder', 'geeksforgeeks'] 
print(check(sentence, words)) 

['coder in geeksforgeeks']
